Load a large GeoTIFF file stored on S3 as out-db raster, and split it into smaller tiles.

Run RS_Value using a DataFrame of points on a large out-db raster.

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import expr, col, lit
from sedona.spark import *

In [2]:
config = SedonaContext.builder().appName('havasu-iceberg-outdb-raster-etl')\
    .config("spark.hadoop.fs.s3a.bucket.wherobots-examples.aws.credentials.provider","org.apache.hadoop.fs.s3a.AnonymousAWSCredentialsProvider")\
    .getOrCreate()
sedona = SedonaContext.create(config)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


Load Raster

In [3]:
raster_df = sedona.sql("SELECT RS_FromPath('s3://wherobots-examples/data/ppp_2020_1km_Aggregated.tif') as rast")
raster_df.show(5)

+--------------------+
|                rast|
+--------------------+
|LazyLoadOutDbGrid...|
+--------------------+



In [4]:
sedona.sql("CREATE NAMESPACE IF NOT EXISTS wherobots.test_db")
sedona.sql("DROP TABLE IF EXISTS wherobots.test_db.world_pop")
raster_df.writeTo("wherobots.test_db.world_pop").create()

In [5]:
sedona.sql("SELECT RS_Metadata(rast) meta FROM wherobots.test_db.world_pop").show(5, False)

+-----------------------------------------------------------------------------------------------------------+
|meta                                                                                                       |
+-----------------------------------------------------------------------------------------------------------+
|{-180.001249265, 83.99958319871001, 43200, 18720, 0.0083333333, -0.0083333333, 0.0, 0.0, 4326, 1, 256, 256}|
+-----------------------------------------------------------------------------------------------------------+



Split raster into tiles
Large rasters may not be suitable for performing raster processing tasks that reads all the pixel data. WherobotsDB provides RS_TileExplode function for splitting the large raster into smaller tiles. When the input raster is an out-db raster, the generated tiles are out-db rasters referencing different parts of the out-db raster file. This is a pure geo-referencing metadata operation so this is very fast.

The tiles generated by RS_TileExplode are within their original partition, so all the tiles are within one partition because the original DataFrame has only one row. This dataframe needs to be repartitioned to distribute the tiles to multiple executors, otherwise future processing on these tiles won't be parallelised.

In [6]:
tile_df = sedona.sql("SELECT RS_TileExplode(rast, 256, 256) AS (x, y, tile) FROM wherobots.test_db.world_pop").repartition(16)
tile_df.show(5)

+---+---+--------------------+
|  x|  y|                tile|
+---+---+--------------------+
|150| 55|OutDbGridCoverage...|
|139| 37|OutDbGridCoverage...|
|146| 31|OutDbGridCoverage...|
| 31| 14|OutDbGridCoverage...|
|131|  2|OutDbGridCoverage...|
+---+---+--------------------+
only showing top 5 rows



Load raster as tiles

In [7]:
raster_df_tiled = sedona.read.format("raster").option("tileWidth", "256").option("tileHeight", "256").load("s3://wherobots-examples/data/ppp_2020_1km_Aggregated.tif")
raster_df_tiled.show(5)

+--------------------+---+---+--------------------+
|                rast|  x|  y|                name|
+--------------------+---+---+--------------------+
|OutDbGridCoverage...| 39| 66|ppp_2020_1km_Aggr...|
|OutDbGridCoverage...| 58| 58|ppp_2020_1km_Aggr...|
|OutDbGridCoverage...| 73| 30|ppp_2020_1km_Aggr...|
|OutDbGridCoverage...|  3| 49|ppp_2020_1km_Aggr...|
|OutDbGridCoverage...|139| 16|ppp_2020_1km_Aggr...|
+--------------------+---+---+--------------------+
only showing top 5 rows



Rename the raster column rast as tile before saving the DataFrame into Havasu table.

In [8]:
tile_df = raster_df_tiled.select(col("rast").alias("tile"), "x", "y")

Saving as out-db rasters

In [9]:
sedona.sql("DROP TABLE IF EXISTS wherobots.test_db.world_pop_tiles")
tile_df.writeTo("wherobots.test_db.world_pop_tiles").create()

In [10]:
sedona.table("wherobots.test_db.world_pop_tiles").count()

12506

Saving tiles as in-db rasters
WherobotsDB provides an RS_AsInDb function for converting out-db raster as in-db raster. 
It needs to fetch all the band data from the raster file.
Manually repartition the out-db tile dataset to run this convertion with high parallelism.

In [11]:
indb_tile_df = tile_df.withColumn("tile", expr("RS_AsInDb(tile)"))
indb_tile_df.show(5)

+--------------------+---+---+
|                tile|  x|  y|
+--------------------+---+---+
|GridCoverage2D["g...| 63| 72|
|GridCoverage2D["g...| 82| 72|
|GridCoverage2D["g...|113| 26|
|GridCoverage2D["g...| 78| 73|
|GridCoverage2D["g...| 18| 62|
+--------------------+---+---+
only showing top 5 rows



In [12]:
sedona.sql("DROP TABLE IF EXISTS wherobots.test_db.world_pop_indb_tiles")
indb_tile_df.writeTo("wherobots.test_db.world_pop_indb_tiles").create()

In [13]:
sedona.table("wherobots.test_db.world_pop_indb_tiles").count()

12506

Visualize the tile boundaries on a map

In [14]:
sedona.table("wherobots.test_db.world_pop_indb_tiles").show()
tiledMap = SedonaKepler.create_map()
SedonaKepler.add_df(tiledMap, sedona.table("wherobots.test_db.world_pop_indb_tiles").withColumn("tile", expr("RS_Envelope(tile)")), name="tiles")
tiledMap

+--------------------+---+---+
|                tile|  x|  y|
+--------------------+---+---+
|GridCoverage2D["h...|100| 61|
|GridCoverage2D["h...|128| 30|
|GridCoverage2D["h...| 77|  8|
|GridCoverage2D["h...|132| 22|
|GridCoverage2D["h...| 15| 60|
|GridCoverage2D["h...| 38| 16|
|GridCoverage2D["h...| 33| 66|
|GridCoverage2D["h...| 58| 46|
|GridCoverage2D["h...|163|  3|
|GridCoverage2D["h...| 64| 48|
|GridCoverage2D["h...|140| 53|
|GridCoverage2D["h...| 38| 39|
|GridCoverage2D["h...| 90| 62|
|GridCoverage2D["h...| 64| 23|
|GridCoverage2D["h...|137| 12|
|GridCoverage2D["h...| 15| 41|
|GridCoverage2D["h...|133|  2|
|GridCoverage2D["h...| 21|  2|
|GridCoverage2D["h...| 53| 38|
|GridCoverage2D["h...| 55|  9|
+--------------------+---+---+
only showing top 20 rows

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


KeplerGl(data={'tiles': {'index': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 2…

Population of POI
Join the POI dataset with the population dataset to evaluate the population of POIs.

Load POI Dataset

In [15]:
spatialRdd = ShapefileReader.readToGeometryRDD(sedona.sparkContext, "s3://wherobots-examples/data/ne_50m_airports")
poi_df = Adapter.toDf(spatialRdd, sedona)
poi_df.show(5)

+--------------------+---------+----------+-----+----------------+------+--------+--------+---------+--------------------+---------+
|            geometry|scalerank|featurecla| type|            name|abbrev|location|gps_code|iata_code|           wikipedia|natlscale|
+--------------------+---------+----------+-----+----------------+------+--------+--------+---------+--------------------+---------+
|POINT (113.935016...|        2|   Airport|major| Hong Kong Int'l|   HKG|terminal|    VHHH|      HKG|http://en.wikiped...|  150.000|
|POINT (121.231370...|        2|   Airport|major|         Taoyuan|   TPE|terminal|    RCTP|      TPE|http://en.wikiped...|  150.000|
|POINT (4.76437693...|        2|   Airport|major|        Schiphol|   AMS|terminal|    EHAM|      AMS|http://en.wikiped...|  150.000|
|POINT (103.986413...|        2|   Airport|major|Singapore Changi|   SIN|terminal|    WSSS|      SIN|http://en.wikiped...|  150.000|
|POINT (-0.4531566...|        2|   Airport|major| London Heathrow|   

Joining POIs with out-db raster
Cartesian join with the single row large out-db raster table, and evaluate the population value on each point.

In [16]:
res_df = poi_df.join(sedona.table("wherobots.test_db.world_pop")).withColumn("pop", expr("RS_Value(rast, geometry)")).drop("rast")
res_df.show(5)
res_df.where("pop > 100").count()

+--------------------+---------+----------+-----+----------------+------+--------+--------+---------+--------------------+---------+------------------+
|            geometry|scalerank|featurecla| type|            name|abbrev|location|gps_code|iata_code|           wikipedia|natlscale|               pop|
+--------------------+---------+----------+-----+----------------+------+--------+--------+---------+--------------------+---------+------------------+
|POINT (113.935016...|        2|   Airport|major| Hong Kong Int'l|   HKG|terminal|    VHHH|      HKG|http://en.wikiped...|  150.000| 1627.572998046875|
|POINT (121.231370...|        2|   Airport|major|         Taoyuan|   TPE|terminal|    RCTP|      TPE|http://en.wikiped...|  150.000|1459.4176025390625|
|POINT (4.76437693...|        2|   Airport|major|        Schiphol|   AMS|terminal|    EHAM|      AMS|http://en.wikiped...|  150.000|1093.3812255859375|
|POINT (103.986413...|        2|   Airport|major|Singapore Changi|   SIN|terminal|    WS

204

Joining POIs with out-db tiles
Spatial join using the POI and out-db raster tile dataset, and evaluates the population value on each point.

In [17]:
res_df = poi_df.join(sedona.table("wherobots.test_db.world_pop_tiles"), expr("RS_Intersects(tile, geometry)")).withColumn("pop", expr("RS_Value(tile, geometry)")).drop("tile")
res_df.show(5)
res_df.where("pop > 100").count()

+--------------------+---------+----------+-----+----------------+------+--------+--------+---------+--------------------+---------+---+---+------------------+
|            geometry|scalerank|featurecla| type|            name|abbrev|location|gps_code|iata_code|           wikipedia|natlscale|  x|  y|               pop|
+--------------------+---------+----------+-----+----------------+------+--------+--------+---------+--------------------+---------+---+---+------------------+
|POINT (113.935016...|        2|   Airport|major| Hong Kong Int'l|   HKG|terminal|    VHHH|      HKG|http://en.wikiped...|  150.000|137| 28| 1627.572998046875|
|POINT (121.231370...|        2|   Airport|major|         Taoyuan|   TPE|terminal|    RCTP|      TPE|http://en.wikiped...|  150.000|141| 27|1459.4176025390625|
|POINT (4.76437693...|        2|   Airport|major|        Schiphol|   AMS|terminal|    EHAM|      AMS|http://en.wikiped...|  150.000| 86| 14|1093.3812255859375|
|POINT (103.986413...|        2|   Airpo

204

Joining POIs with in-db tiles